In [3]:
#ライブラリ読み込み
import os
import sys  
from google.colab import drive  
drive.mount('/content/drive')   
import random
import torch.backends.cudnn as cudnn
import torch
import numpy as np
import pandas as pd
import scipy.spatial
from transformers import BertJapaneseTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util
import ipadic
import gc
gc.collect()
torch.cuda.empty_cache()

Mounted at /content/drive


# データ読み込み

In [4]:
DATA_IN_PATH = "/content/drive/My Drive/Colab Notebooks/"
train=pd.read_csv(DATA_IN_PATH + 'data_complete_34814.csv',header=0,encoding="cp932")

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)



#Sentence－BERTローディング
MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2" 
model = SentenceBertJapanese(MODEL_NAME)

#レビューデータをリスト化する
data=train['review_cleansing']
data=pd.Series(data)
data=data.tolist()
b=[]


Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/667 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

# レビューデータをembeddingベクトルに変換

### レビューデータをembeddingベクトルに変換

In [ ]:
for i in data[34000:]:

  i=pd.Series(i)
  i=i.tolist() 
  sentence_embeddings = model.encode(i)
  b.append(sentence_embeddings)
  result = torch.cat(b, dim=0)
  result=result.cpu()
  result=result.detach().numpy()

print(result)
print(len(result))
np.save('/content/035.npy', result)

### 設問項目をembeddingベクトルに変換

In [5]:
def convert_npy(a):
  q=pd.read_csv(DATA_IN_PATH + a + '.csv',header=0,encoding="cp932")
  m=model.encode(q['q'])
  m=m.detach().numpy()
  np.save('/content/'+ a + '.npy', m)

  return  print(len(m)),print(m)

for a in survey:
  convert_npy(a)

['JOBSF', 'ENGAGEMENT', 'TURNOVER']
3
3
[[-0.7070186   0.54293275  0.02849811 ...  0.9439904  -0.47870636
  -1.0452493 ]
 [-1.1237226   0.67608076 -0.13215558 ...  0.99938864 -0.33469167
  -0.57112586]
 [-0.39877152  0.39090428  0.02124195 ...  0.9922373  -0.703538
  -0.6284645 ]]
7
[[-0.18690181  0.49394786  0.43271434 ...  1.9212064  -1.0788887
  -1.1677817 ]
 [-0.8378063   0.85018647  0.30319294 ...  0.58584344 -0.45065483
  -0.8557275 ]
 [ 0.24815339  0.495097   -0.15647258 ...  0.666002   -1.3703095
  -0.8321519 ]
 ...
 [ 0.7532132   0.33607233 -0.41836065 ...  0.42347682 -0.20178254
  -0.87566406]
 [-1.258475    0.54188174  0.3617952  ...  0.90320826 -0.06636568
  -0.16057944]
 [-0.3056361   0.7964192   0.3756936  ...  1.0352064   0.10456257
  -0.7767802 ]]
2
[[ 0.31608918 -0.2362833  -0.15508266 ...  0.27913433  0.01208632
   0.09116961]
 [-0.27815711 -0.36280826  0.11036822 ...  0.3395453  -0.15054817
  -0.20841165]]
